In [38]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve
import numpy as np
import plotly.express as px
# import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm
from catboost.utils import get_roc_curve, select_threshold
from torchmetrics.classification import BinaryPrecisionRecallCurve
# from supervised.automl import AutoML
import warnings
from loguru import logger
from sklearn.metrics import f1_score, precision_score, recall_score
from catboost.utils import eval_metric
from catboost import metrics as cat_metrics
from sklearn.utils import class_weight

warnings.filterwarnings("ignore")


In [39]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)
multilabelDf.columns

Index(['vision_bonnet', 'vision_bumper_front', 'vision_engine',
       'vision_grille', 'vision_headlamp_lh', 'vision_headlamp_rh',
       'vision_door_front_lh', 'vision_door_front_rh', 'vision_bumper_rear',
       'vision_misc', 'vision_front_panel', 'vision_non_external',
       'vision_wheel', 'vision_fender_front_lh', 'vision_fender_front_rh',
       'vision_rear_quarter_lh', 'vision_tail_lamp_lh', 'vision_tail_lamp_rh',
       'vision_windscreen_front', 'vision_rear_compartment',
       'vision_rear_panel', 'vision_rear_quarter_rh', 'vision_door_rear_rh',
       'vision_door_rear_lh', 'vision_windscreen_rear', 'bonnet',
       'bumper_front', 'air_conditioning', 'cooling_fan', 'grille',
       'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh',
       'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel',
       'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh',
       'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh',
       

In [40]:
# multilabelDf.replace(pd.NA, 0, inplace=True)

In [41]:
multilabelDf["Assembly_Type"]

0         3
1         3
2         3
3         3
4         3
         ..
595704    3
595705    3
595706    2
595707    3
595708    3
Name: Assembly_Type, Length: 595709, dtype: Int64

In [42]:
multilabelDf[multilabelDf["Model"] == "NA"]

,vision_bonnet,vision_bumper_front,vision_engine,vision_grille,vision_headlamp_lh,vision_headlamp_rh,vision_door_front_lh,vision_door_front_rh,vision_bumper_rear,vision_misc,...,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Claim_Type,Vehicle_Type,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer


In [43]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
        path=f"s3://multilabel_df/",
        dataset=True,
    )
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
    "Model",
    # "Vehicle_Still_Driveable",
    # "NCB_Stat",
    # "Assembly_Type",
    # "Claim_Type",
    "Vehicle_Type",
    # # "Sum_Insured",
    # "Repairer",
    # # # "Repairer_Apprv_Count",
    # "Collision_With",
    # "Handling_Insurer",
]
exclude = ['Sum_Insured', 'Repairer', 'Repairer_Apprv_Count', 
"Vehicle_Still_Driveable",
    "NCB_Stat",
    "Assembly_Type",
    "Claim_Type",
    # "Vehicle_Type",
    "Sum_Insured",
    "Repairer",
    "Repairer_Apprv_Count",
    "Collision_With",
    "Handling_Insurer"
     ]
# contFeatures = ["Sum_Insured", "Repairer_Apprv_Count"]
# allTargetCol = [
#    'bonnet', 
#    'bumper_front', 
#    'grille', 
#    'fog_lamp_rh', 
#    'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 
#    'air_conditioning', 
#    'cooling_fan', 
#    'radiator', 
#    'bumper_rear', 'misc', 
#    'engine', 
#    'airbag', 
#    'front_panel', 
#    'undercarriage',
#     'wheel', 
#     'fender_front_lh', 
#     'fender_front_rh',
#      'door_mirror_rh', 
#      'rear_quarter_lh',
#       'interior',
#        'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 
#        'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
# ]

allTargetCol = [
    'fog_lamp_rh', 
    'fog_lamp_lh', 
    'air_conditioning', 
    'undercarriage',
    'airbag',
    'cooling_fan', 
    'radiator',
    'interior',
   'bonnet', 
   'bumper_front', 
   'grille', 
   'headlamp_lh',
   'headlamp_rh', 
   'bumper_rear', 
   'engine', 
   'front_panel', 
   'fender_front_lh',
   'fender_front_rh', 
   'rear_quarter_lh', 
   'tail_lamp_lh', 
   'tail_lamp_rh',
   'windscreen_front', 
   'rear_compartment',
   'rear_panel', 
   'rear_quarter_rh', 
   'windscreen_rear'
]
allVisionFeatures = [
    'vision_bonnet',
    'vision_bumper_front',
    'vision_engine',
    'vision_grille',
    'vision_headlamp_lh',
    'vision_headlamp_rh',
    'vision_bumper_rear',
    'vision_front_panel',
    'vision_non_external',
    'vision_fender_front_lh',
    'vision_fender_front_rh',
    'vision_rear_quarter_lh',
    'vision_tail_lamp_lh',
    'vision_tail_lamp_rh',
    'vision_windscreen_front',
    'vision_rear_compartment',
    'vision_rear_panel',
    'vision_rear_quarter_rh',
    'vision_windscreen_rear',

    ]
allInputFeature = caseFeatures + allVisionFeatures


In [44]:
expId = 126
saloonPredDf = pd.read_csv(
    f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/Saloon - 4 Dr/Saloon - 4 Dr_imgs_pred_output.csv"
)
hatchBackDf = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/Hatchback - 5 Dr/Hatchback - 5 Dr_imgs_pred_output.csv")
suvDf = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/SUV - 5 Dr/SUV - 5 Dr_imgs_pred_output.csv")

imgPredOutput = pd.concat([saloonPredDf, hatchBackDf, suvDf])
# imgPredOutput = saloonPredDf
allPredModel = []
allPreds = []
allGt = []
realTestDataDf = multilabelDf[caseFeatures + allTargetCol + ["CaseID"]].merge(
    imgPredOutput, on="CaseID"
)
realTestDataDf = realTestDataDf.loc[
    :, ~realTestDataDf.columns.str.contains("^Unnamed")
]
trainDf = multilabelDf[
    ~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())
]
# trainDf = trainDf.sample(frac=0.5)
assert set(realTestDataDf["CaseID"].tolist()).isdisjoint(trainDf["CaseID"].tolist())
targetPart = [
    "fog_lamp_rh",
    "airbag",
    "undercarriage",
    "door_mirror_rh",
    "interior",
    "fog_lamp_lh",
    "door_mirror_lh",
    "windscreen_rear",
    "air_conditioning",
    "cooling_fan",
    "radiator"
]
# targetPart = [
#      'bonnet', 'bumper_front', 'grille', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'bumper_rear', 'misc', 'engine', 'front_panel', 'wheel', 'fender_front_lh', 'fender_front_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_rear_lh'
# ]
# targetCol = list(set(targetPart) & set(allTargetCol))
targetCol = allTargetCol


In [45]:
allVisionFeatures

['vision_bonnet',
 'vision_bumper_front',
 'vision_engine',
 'vision_grille',
 'vision_headlamp_lh',
 'vision_headlamp_rh',
 'vision_bumper_rear',
 'vision_front_panel',
 'vision_non_external',
 'vision_fender_front_lh',
 'vision_fender_front_rh',
 'vision_rear_quarter_lh',
 'vision_tail_lamp_lh',
 'vision_tail_lamp_rh',
 'vision_windscreen_front',
 'vision_rear_compartment',
 'vision_rear_panel',
 'vision_rear_quarter_rh',
 'vision_windscreen_rear']

In [46]:

for part in tqdm(targetCol):
    logger.success(f"Start training for part : {part}")
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []
    trainCaseId = trainDf["CaseID"].tolist()
    tempTestDataDf = realTestDataDf
    # tempTestDataDf = realTestDataDf.groupby(part).head(5000)
    testCaseId = tempTestDataDf["CaseID"].tolist()
    # sampledDf = trainDf[allInputFeature + [part]].groupby(part).sample(frac=0.3)
    X_train = trainDf[allInputFeature]
    Y_train = trainDf[part].to_frame()
   
    X_test = tempTestDataDf[allInputFeature]
    Y_test = tempTestDataDf[part].to_frame()

    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    neg_count = len(Y_test[Y_test[part] == 0]) / len(Y_test)
    pos_weight = neg_count / pos_count
    train_pool = Pool(
        X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
    )
    test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
    clsWeight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y_train.values),y=Y_train.values.squeeze(1))
    logger.success(f"train_class_weight : {clsWeight}")

    testClsWeights = {
        0 : clsWeight[0],
        1 : clsWeight[1]

    }
    print(testClsWeights)
    sampleWeights = class_weight.compute_sample_weight(class_weight=testClsWeights, y=Y_test.values)
    print(f"Shape :{sampleWeights.shape}")
    # prCurve = MulticlassPrecisionRecallCurve(num_classes=2, thresholds=11)
    clf = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="F1",
            iterations=300,
            early_stopping_rounds=100,
            task_type="GPU",
            # class_weights= clsWeight,
            auto_class_weights="Balanced",
            use_best_model=True,
            # scale_pos_weight=20
            # class_names=["not_dmg", "dmg"],
        )
    clf.fit(train_pool, eval_set=test_pool, metric_period=50, plot=False, verbose=1)
    test_predict = clf.predict(test_pool)
    acc = accuracy_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    confMat = confusion_matrix(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    pos_count_abs = len(Y_test[Y_test[part] == 1])
    # catboostF1 = cat_metrics.F1(Y_test.values.astype(np.int64), test_predict)
    tn = confMat[0][0]
    tp = confMat[1][1]
    fp = confMat[0][1]
    fn = confMat[1][0]
    totalSample = fp + fn + tp + tn
    acc = (tp + tn) / (fp + fn + tp + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  (2 * precision * recall) / (precision + recall)
    f1_weighted = f1_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # precision_weighted = precision_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # recall_weighted = recall_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)

    logger.success(f"F1 for part : {part} = {f1}")
    logger.success(f"Weighted F1 scikit for part : {part} = {f1_weighted}")

    # assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
    #     fp / (tn + fp)
    # )

    allPredModel.append(
        {
            "part": part,
            "tp": tp / (tp + fn),
            "tn": tn / (tn + fp),
            "fp": fp / (tn + fp),
            "fn": fn / (tp + fn),
            "acc": acc,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "pos_count": pos_count,
            "pos_count_abs": pos_count_abs,

        }
    )
    assert len(testCaseId) == len(test_predict)
    assert len(testCaseId) == len(Y_test)

    allPreds.append({"CaseID": testCaseId, part: test_predict})
    allGt.append({"CaseID": testCaseId, part: Y_test.values.squeeze(1)})


  0%|          | 0/26 [00:00<?, ?it/s]2023-01-05 02:48:01.357 | SUCCESS  | __main__:<module>:2 - Start training for part : fog_lamp_rh
2023-01-05 02:48:02.484 | SUCCESS  | __main__:<module>:25 - train_class_weight : [0.55091834 5.40982257]


{0: 0.5509183369346546, 1: 5.409822571794403}
Shape :(4199,)
Learning rate set to 0.068137
0:	learn: 0.8748981	test: 0.6930101	best: 0.6930101 (0)	total: 30.7ms	remaining: 9.18s
50:	learn: 0.8937171	test: 0.7098210	best: 0.7116890 (41)	total: 1.68s	remaining: 8.18s
100:	learn: 0.8944772	test: 0.7061778	best: 0.7116890 (41)	total: 3.24s	remaining: 6.38s


2023-01-05 02:48:07.429 | SUCCESS  | __main__:<module>:68 - F1 for part : fog_lamp_rh = 0.7116890149091576
2023-01-05 02:48:07.429 | SUCCESS  | __main__:<module>:69 - Weighted F1 scikit for part : fog_lamp_rh = 0.7116890149091561
  4%|▍         | 1/26 [00:06<02:31,  6.07s/it]2023-01-05 02:48:07.430 | SUCCESS  | __main__:<module>:2 - Start training for part : fog_lamp_lh


bestTest = 0.7116890131
bestIteration = 41
Shrink model to first 42 iterations.


2023-01-05 02:48:08.529 | SUCCESS  | __main__:<module>:25 - train_class_weight : [0.54266972 6.35895506]


...

100:	learn: 1.0000000	test: 0.7425115	best: 0.7425115 (0)	total: 2.58s	remaining: 5.08s
bestTest = 0.7425115387
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-05 02:50:13.233 | SUCCESS  | __main__:<module>:25 - train_class_weight : [ 0.5212442  12.26791936]


{0: 0.5212441972358223, 1: 12.267919362867097}
Shape :(4199,)
Learning rate set to 0.068137
0:	learn: 1.0000000	test: 0.6178812	best: 0.6178812 (0)	total: 22.8ms	remaining: 6.81s
50:	learn: 1.0000000	test: 0.6178812	best: 0.6178812 (0)	total: 1.25s	remaining: 6.11s


2023-01-05 02:50:16.111 | SUCCESS  | __main__:<module>:68 - F1 for part : windscreen_rear = 0.6178812202889105
2023-01-05 02:50:16.111 | SUCCESS  | __main__:<module>:69 - Weighted F1 scikit for part : windscreen_rear = 0.6178812202889099
100%|██████████| 26/26 [02:14<00:00,  5.18s/it]

100:	learn: 1.0000000	test: 0.6178812	best: 0.6178812 (0)	total: 2.49s	remaining: 4.91s
bestTest = 0.6178812204
bestIteration = 0
Shrink model to first 1 iterations.


In [47]:

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgFp = evalMetrics["fp"].mean()

avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()
avgPosCount = evalMetrics["pos_count"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")
print(f"Avg pos_count : {avgPosCount}")

print(f"avgAcc : {avgAcc}")

Avg Precision : 0.8496287017415083
Avg Recall : 0.6449505947069958
Avg F1 : 0.7270279667227866
Avg TP : 0.6449505947069958
Avg TN : 0.8996826929367241
Avg FN : 0.35504940529300416
Avg pos_count : 0.18743473720849285
avgAcc : 0.7876067481754376


In [48]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count_abs', text_auto=True)
fig

# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'array',
#         tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
#     )
# )

In [49]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count')

fig.add_hline(y=avgPosCount, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)

In [50]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [51]:
fig = px.bar(evalMetrics, x = 'part', y = 'fp')

fig.add_hline(y=avgFp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFp, 2), 0.2, 0.5, 0.7],
    )
)

In [52]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [53]:
fig = px.bar(evalMetrics, x = 'part', y = 'fn')

fig.add_hline(y=avgFn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [54]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgAcc, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [55]:
fig = px.bar(evalMetrics, x = 'part', y = 'precision')

fig.add_hline(y=avgPrecision, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgPrecision, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [56]:
fig = px.bar(evalMetrics, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [57]:
fig = px.bar(evalMetrics, x = 'part', y = 'f1')

fig.add_hline(y=avgF1, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgF1, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [58]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 25/25 [00:00<00:00, 134.47it/s]


,CaseID,fog_lamp_rh,fog_lamp_lh,air_conditioning,undercarriage,airbag,cooling_fan,radiator,interior,bonnet,...,fender_front_lh,fender_front_rh,rear_quarter_lh,tail_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,windscreen_rear
247,10000064,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,0,0,0,0
1534,10001809,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1535,10002135,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
248,10003477,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1536,10003541,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,13548803,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3375,13558029,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1965,13558341,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3376,13561670,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [59]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)

100%|██████████| 25/25 [00:00<00:00, 127.42it/s]


In [60]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,fog_lamp_rh,fog_lamp_lh,air_conditioning,undercarriage,airbag,cooling_fan,radiator,interior,bonnet,...,fender_front_lh,fender_front_rh,rear_quarter_lh,tail_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,windscreen_rear
247,10000064,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1534,10001809,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1535,10002135,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
248,10003477,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1536,10003541,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,13548803,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3375,13558029,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1965,13558341,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3376,13561670,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = np.divide(len(set_true.intersection(set_pred)), 
                    float( len(set_true) ))
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean( [x for x in acc_list if not np.isnan(x)]), acc_list

In [62]:
# tempTargetCol = ["airbag", "undercarriage", "interior"]
from sklearn.metrics import hamming_loss

In [63]:
hammingLoss = hamming_loss(multilabelGt[targetCol].values.astype(np.int32), multilabelPred[targetCol].values)
subsetAcc2 = 1 - hammingLoss

In [64]:
evalData = pd.DataFrame(
    [
        {
            "metric" : "subset_accuracy",
            "value" : subsetAcc2

        },
         {
            "metric" : "avg_f1",
            "value" : avgF1

        },
         {
            "metric" : "avg_tpr",
            "value" : avgTp

        },
        {
            "metric" : "avg_tnr",
            "value" : avgTn

        },
        {
            "metric" : "part_recall",
            "value" : avgRecall

        },
            {
            "metric" : "part_precision",
            "value" : avgPrecision

        },
                {
            "metric" : "part_accuracy",
            "value" : avgAcc

        }
    ]
)


In [65]:
fig = px.bar(evalData, x = 'metric', y = 'value', text_auto=True)
fig

In [66]:
corr = evalMetrics["tp"].corr(evalMetrics["pos_count"])
corr

0.33991374350617076

In [67]:
corr = evalMetrics["f1"].corr(evalMetrics["pos_count"])
corr

0.3350640177835273

In [68]:
corr = evalMetrics["acc"].corr(evalMetrics["pos_count"])
corr

-0.01343463952539425

In [69]:
targetCol[-1]

'windscreen_rear'

In [70]:
clf.classes_

array([0, 1])

In [71]:
# explainer = shap.Explainer(clf)
# shap_values = explainer(X_test)
# plt.title(targetCol[-1])
# shap.plots.beeswarm(shap_values)


In [72]:
caseStudyCases = multilabelPred.sample(n=30)["CaseID"].tolist()

In [73]:
caseStudyPred = multilabelPred[multilabelPred['CaseID'].isin(caseStudyCases)]
caseStudyGt = multilabelGt[multilabelGt["CaseID"].isin(caseStudyCases)]
caseStudyPred.to_csv("case_study_pred.csv")
caseStudyGt.to_csv("case_study_gt.csv")

In [74]:
caseStudyPred

,CaseID,fog_lamp_rh,fog_lamp_lh,air_conditioning,undercarriage,airbag,cooling_fan,radiator,interior,bonnet,...,fender_front_lh,fender_front_rh,rear_quarter_lh,tail_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,windscreen_rear
3672,10047111,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3090,10049081,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
3888,10052907,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3456,10121129,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2707,10201297,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2523,10211376,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3694,10212861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
438,10230343,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3235,10281174,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2319,10314722,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
